In [70]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html, Dash
from dash.dependencies import Input, Output
import plotly.graph_objects as go

import plotly.express as px

In [71]:
df = pd.read_csv('Data/homicide_data.csv')

us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")

In [72]:
df['victim_full_name'] = df['victim_first'] +  ' ' + df['victim_last']

df = df.sort_values('state')

In [79]:
def make_map(state):
    filtered_df = df[df['state'] == state]
    fig = px.scatter_mapbox(
        filtered_df,
        lat="lat",
        lon="lon",
        hover_name="victim_full_name",
        hover_data=["victim_race", "victim_age", "victim_sex"],
        color = "victim_race",
        #color_discrete_sequence=["blue"],
        zoom=7,
        height=500,
    )
    fig.update_layout(mapbox_style="open-street-map", title = f'Murders in {state}')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    #fig.update_layout(mapbox_bounds={"west": -180, "east": -50, "south": 20, "north": 90})
    fig.update_geos(fitbounds="locations")


    
    return fig

app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    dcc.Dropdown(
        id='state-dropdown',
        options=[
            {'label': state, 'value': state} for state in df['state'].unique()
        ],
        value=df['state'].unique()[0],  # Set the default value
    ),
    dcc.Graph(id='murder-map')
])

# Define callback to update the map when dropdown selection changes
@app.callback(
    Output('murder-map', 'figure'),
    [Input('state-dropdown', 'value')]
)
def update_map(selected_state):
    fig = make_map(selected_state)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

